In [1]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
# from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import openai
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
import requests
import os
from datetime import datetime
from time import sleep
import pandas as pd
import faiss

In [2]:
os.environ["OPENAI_KEY"] = "***"
os.environ["AZURE_OPENAI_KEY"] = "***"
os.environ["AZURE_OPENAI_API_KEY"] = "***"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lab-ai-openai-eus.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

In [3]:
client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01" #2024-02-15-preview ?
)

message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."}]

completion = client.chat.completions.create(
  model="gpt-35-turbo", # model = "deployment_name"
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

print(completion)

ChatCompletion(id='chatcmpl-9WKPLMNDuVbY3gXxqNCR05lwg44Sx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I am an AI assistant and I'm here to help you find the information you need. Please let me know what you're looking for and I'll do my best to assist you.", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1717491971, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=39, prompt_tokens=19, total_tokens=58), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity

In [4]:
llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment="gpt-35-turbo",
)

from langchain_core.messages import HumanMessage

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

AIMessage(content="J'aime programmer.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-a696ddf5-dd63-47c4-b5e5-4ab7de86b4ed-0', usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25})

In [5]:
# !pip install unstructured
# !pip install "unstructured[pdf]"

In [6]:
# upload pdf 

from langchain_community.document_loaders import DirectoryLoader

directory_path = "microsoft_learn"

loader = DirectoryLoader(directory_path)
pages = loader.load_and_split()
pages

# partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"

[Document(page_content="Le service Azure OpenAI : un outil puissant pour l'intelligence artificielle\n\nUn document qui présente le service Azure OpenAI, ses caractéristiques, ses bénéfices et ses applications.\n\nQu'est-ce que le service Azure OpenAI ? Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI, une initiative qui vise à créer une intelligence artificielle générale (AGI) bénéfique pour l'humanité. Le service Azure OpenAI offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés.\n\nComment fonctionne le service Azure OpenAI ? Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI. Le portail Azure OpenAI est une interface web qui permet de créer, de gérer et de surveiller les ressources du service Azure OpenAI, telles q

RAG : recherche par similarité (en utilisant les embeddings)

Extract text from pdf 

In [7]:
#!pip install pdfminer.six

In [8]:
from pdfminer.high_level import extract_text

dirs = os.listdir(directory_path)
output = "pdftotext.txt"
# supprimer le fichier s'il existe déjà
if os.path.exists(output):  
    os.remove(output) 
for file in dirs:
    path_file = os.path.join(directory_path, file)
    print(f"PDF path file: {path_file}")
    text = extract_text(path_file)
    with open(output, 'a', encoding='utf-8') as fichier:
        fichier.write(text)


PDF path file: microsoft_learn\Le service Azure OpenAI.pdf


In [9]:
from langchain.document_loaders import TextLoader, DirectoryLoader

loader = TextLoader("pdftotext.txt", encoding='utf-8')
documents= loader.load()
len(documents)

# 'charmap' codec can't decode byte 0x9d in position 3802475: character maps to <undefined>

1

In [10]:
documents

[Document(page_content="Le service Azure OpenAI : un outil puissant \npour l'intelligence artificielle \n\nUn document qui présente le service Azure OpenAI, ses caractéristiques, ses bénéfices et \nses applications. \n\nQu'est-ce que le service Azure OpenAI ? \nLe service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à \nla plateforme OpenAI, une initiative qui vise à créer une intelligence artificielle générale (AGI) bénéfique \npour l'humanité. Le service Azure OpenAI offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou \npersonnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des \nscénarios variés. \n\nComment fonctionne le service Azure OpenAI ? \nLe service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API \nAzure OpenAI. Le portail Azure OpenAI est une interface web qui permet de créer, de gérer et de \nsurveiller les ressources du service

In [11]:
# chunk settings  
chunk_size = 200
chunk_overlap = 30 # duplication des informations entre les vecteurs coupés 

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(documents)

print(len(chunks)) # 35 vecteurs

35


In [13]:
chunks[34].page_content

'•  Analyser les résultats des requêtes, en utilisant les outils de suivi et de statistiques fournis par le \n\nportail Azure OpenAI.'

In [14]:
#from langchain.embeddings import OpenAIEmbeddings
#import numpy as np
#from langchain.vectorstores import Chroma


#embeddings = OpenAIEmbeddings(deployment_id="text-embedding-ada-002",chunk_size="100")

#docsearch = Chroma.from_documents(chunks, embeddings)
#for splitted_document in chunks:
 #   Chroma.from_documents(documents=[splitted_document], embedding=embeddings)
  #  time.sleep(60)

In [15]:
# from langchain.embeddings import OpenAIEmbeddings
# from langchain_community.vectorstores import FAISS

# # TODO : The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`

# embeddings = OpenAIEmbeddings(
#     deployment_id="text-embedding-ada-002",
#     api_key=os.getenv("AZURE_OPENAI_KEY"),
#     chunk_size=chunk_size
#     )

# # faiss_vectorstore= FAISS.from_documents(documents, embeddings) # Limite de tokens !!! 


Enregistrer les chunks en tant que vecteur en utilisant faiss  
pour résoudre le problème des tokens appels nombreux API, on découpe les chunks
 en blocs , sleep 
save le tout en local pour ne pas relancer à chaque fois

In [16]:
# !pip install -U langchain-community faiss-cpu langchain-openai tiktoken

In [17]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import AzureOpenAIEmbeddings


embeddings = AzureOpenAIEmbeddings(
    azure_deployment= "text-embedding-ada-002",
    api_key=os.environ["AZURE_OPENAI_KEY"],
)

# Iterate over each chunk of documents
bloc = 40
for i in range(0, len(chunks), bloc):
    print(i)
    faiss_vectorstore= FAISS.from_documents(chunks[i:i+bloc], embeddings)
    sleep(10)
    # print(faiss_vectorstore)

faiss_retriever = faiss_vectorstore.as_retriever()
faiss_vectorstore.save_local("faiss_index")


# TODO : optimiser la taille des chunks plus tard
# LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


0


Retrievers are used to find relevant documents or passages that contain the answer to a given query.
The system "retrieves" any documents that could be relevant in answering the question, and then passes those documents (along with the original question) to the language model for a "generation" step.

La question est posée en utilisant le chat,
toujours en utilisant la similarité sématique 


Test utilisant LCEL


In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


template_context1 =  """answer the question only according to the context  : {context}
            Question: {question}
        """
template_context2= """answer the question only according to the 429 th line and the lines just after of the file  : {context}
            Question: {question}
        """

prompt = ChatPromptTemplate.from_template(template_context1)

retrieval_chain = (
    {"context": faiss_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
question = "quelles sont  les certifications Associate? et quel est leur nombre? "
question = "qu'est ce que le service Azure OpenAI ? "

In [20]:
retrieval_chain.invoke(question)

"Le service Azure OpenAI est présenté comme un outil puissant pour l'intelligence artificielle dans le premier document et est décrit en détail dans les autres documents."

In [21]:
# RetrievalQA
from langchain.chains import RetrievalQA
import faiss
from langchain_openai import AzureChatOpenAI

# TODO : trouver comment load faiss_retriever = index = faiss.read_index( "faiss_index") index.as_retriever()

qa_stuff = RetrievalQA.from_chain_type( #creer une chain pour répondre à la question
    llm=llm, 
    chain_type="stuff", # tous les documents d'un coup et les passe au LLM
    retriever=faiss_retriever, 
    return_source_documents= True,
    verbose=True
)

In [22]:

question = "what is a voucher code?"
question = "Pourquoi utiliser Azure OpenAI ?"
result = qa_stuff.invoke(question)
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Pourquoi utiliser Azure OpenAI ?',
 'result': "Le service Azure OpenAI présente plusieurs avantages pour l'intelligence artificielle, tels que la possibilité de créer des modèles en utilisant les outils de formation et de réglage fournis par le service, ainsi que des caractéristiques telles que la puissance et la flexibilité de la plateforme. Cependant, les raisons spécifiques pour lesquelles une personne ou une entreprise peut choisir d'utiliser Azure OpenAI peuvent varier en fonction de leurs besoins et objectifs en matière d'IA.",
 'source_documents': [Document(page_content='Quels sont les avantages du service Azure OpenAI ? \nLe service Azure OpenAI présente plusieurs avantages, parmi lesquels : \n\n• \n\n• \n\n• \n\n•', metadata={'source': 'pdftotext.txt'}),
  Document(page_content="Le service Azure OpenAI : un outil puissant \npour l'intelligence artificielle \n\nUn document qui présente le service Azure OpenAI, ses caractéristiques, ses bénéfices et \nses applications

Préparer un golden dataset 


In [23]:
questions = ["Quel est le nombre  des certifications Azure ? ","Quel est le nombre des certifications Associate?", "qu est ce qu un voucher en un mot"]
ground_truths = [["Il y a 15 certifications Azure"],
                 ["Il y a 10 certifications Assiociate mentionnees"], 
                 ["une promotion"]
                ]
contexts = []
results = []

In [24]:
questions = [
"Qu'est-ce que le service Azure OpenAI ?",
"Comment fonctionne le service Azure OpenAI ?",
"Quels sont les avantages du service Azure OpenAI ?",
"Quels sont les cas d'utilisation du service Azure OpenAI ?",
"Comment mettre en œuvre le service Azure OpenAI ?",
"Quels sont les composants principaux du service Azure OpenAI ?",
"Quels sont les modèles d'IA accessibles via le service Azure OpenAI ?",
"Quelles tâches les modèles d'IA du service Azure OpenAI peuvent-ils accomplir ?",
"Quelle est l'infrastructure sur laquelle repose le service Azure OpenAI ?",
"Quels outils le service Azure OpenAI fournit-il pour la formation et le réglage des modèles d'IA ?"
]

answers = [  
"Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI, une initiative qui vise à créer une intelligence artificielle générale (AGI) bénéfique pour l'humanité. Il offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés.",   
"Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI. Le portail est une interface web qui permet de créer, de gérer et de surveiller les ressources du service, tandis que les API sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service.",   
"Le service Azure OpenAI présente plusieurs avantages, parmi lesquels l'accès à des modèles d'IA de pointe, une grande flexibilité, une haute performance grâce à l'infrastructure cloud de Microsoft Azure, et une facilité d'intégration grâce à des API simples et des SDK dans différents langages de programmation.",   
"Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité, tels que l'éducation, la santé, le divertissement et le commerce.",   
"Pour mettre en œuvre le service Azure OpenAI, il faut créer un compte Microsoft Azure, souscrire à un abonnement au service, créer une clé d'API, choisir ou créer un modèle d'IA, créer un point de terminaison, envoyer des requêtes au point de terminaison, et analyser les résultats des requêtes.",   
"Les composants principaux du service Azure OpenAI sont le portail Azure OpenAI et les API Azure OpenAI.",   
"Les modèles d'IA accessibles via le service Azure OpenAI incluent GPT-3, Codex, DALL-E et CLIP.",   
"Les modèles d'IA du service Azure OpenAI peuvent réaliser des tâches complexes et variées, telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia.",   
"L'infrastructure sur laquelle repose le service Azure OpenAI est l'infrastructure cloud de Microsoft Azure.",   
"Le service Azure OpenAI fournit des outils de formation et de réglage pour créer son propre modèle d'IA."
]  

ground_truths = [  
["Le service Azure OpenAI est un service cloud qui permet aux développeurs et aux chercheurs d'accéder à la plateforme OpenAI", "Le service Azure OpenAI offre la possibilité d'utiliser des modèles d'IA pré-entraînés ou personnalisés, de les déployer sur le cloud ou en périphérie, et de les intégrer à des applications et des scénarios variés."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Le portail Azure OpenAI est une interface web qui permet de créer, de gérer et de surveiller les ressources du service Azure OpenAI, telles que les abonnements, les clés, les modèles, les points de terminaison et les requêtes."],  
["Le service Azure OpenAI présente plusieurs avantages, parmi lesquels :", "Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure, qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Le service Azure OpenAI peut être utilisé pour de nombreux cas d'utilisation, dans différents domaines et secteurs d'activité.", "Dans le domaine du commerce, le service Azure OpenAI peut être utilisé pour créer des outils d'optimisation, tels que des recommandations personnalisées, des prévisions de ventes, des analyses de marché ou des chatbots."],  
["Pour mettre en œuvre le service Azure OpenAI, il faut suivre les étapes suivantes :", "Analyser les résultats des requêtes, en utilisant les outils de suivi et de statistiques fournis par le portail Azure OpenAI."],  
["Le service Azure OpenAI repose sur deux composants principaux : le portail Azure OpenAI et les API Azure OpenAI.", "Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI, tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Les API Azure OpenAI sont des interfaces de programmation qui permettent d'interagir avec les modèles d'IA du service Azure OpenAI", "tels que GPT-3, Codex, DALL-E ou CLIP."],  
["Il permet d'accéder à des modèles d'IA de pointe, capables de réaliser des tâches complexes et variées", "telles que la génération de texte, la compréhension du langage naturel, la vision par ordinateur, la synthèse vocale ou la création de contenu multimédia."],  
["Il garantit une haute performance, en s'appuyant sur l'infrastructure cloud de Microsoft Azure", "qui assure une disponibilité, une scalabilité et une sécurité optimales."],  
["Choisir un modèle d'IA, parmi ceux proposés par le service Azure OpenAI", "ou créer son propre modèle, en utilisant les outils de formation et de réglage fournis par le service Azure OpenAI."]  
]

results = []
contexts = []

In [25]:
for query in questions:
    result = qa_stuff({"query": query})
    results.append(result['result']) 
    sources = result["source_documents"]
    contents = [source.page_content for source in sources]  

    processed_contents = []
    for content in contents:
        if isinstance(content, str):  
            processed_content = content.split('.')[:2]  
            processed_contents.extend(processed_content)  
        else:
            print(f"Expected a string but got {type(content)}") 
    contexts.append(processed_contents)

print(len(questions))
print(len(ground_truths))
print(len(results))
print(len(contexts))




> Entering new RetrievalQA chain...


c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.
10
10
10
10


In [26]:
# Creating a DataFrame
data = {
    'question': questions,
    'answer': results,
    'contexts': contexts,
    'ground_truth': ground_truths
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

# Save as .csv
df.to_csv('ragas_aoai_dataset.csv', index=False)  

                                            question  \
0            Qu'est-ce que le service Azure OpenAI ?   
1       Comment fonctionne le service Azure OpenAI ?   
2  Quels sont les avantages du service Azure Open...   
3  Quels sont les cas d'utilisation du service Az...   
4  Comment mettre en œuvre le service Azure OpenAI ?   

                                              answer  \
0  Le service Azure OpenAI est un outil pour l'in...   
1  Le service Azure OpenAI repose sur deux compos...   
2  Le texte mentionne que le service Azure OpenAI...   
3  Le document ne fournit pas de détails sur les ...   
4  Pour mettre en œuvre le service Azure OpenAI, ...   

                                            contexts  \
0  [Le service Azure OpenAI : un outil puissant \...   
1  [Comment fonctionne le service Azure OpenAI ? ...   
2  [Quels sont les avantages du service Azure Ope...   
3  [Quels sont les avantages du service Azure Ope...   
4  [Comment mettre en œuvre le service Azure O

In [27]:
# https://blog.cellenza.com/securite-2/utiliser-azure-openai-langchain-et-ragas-pour-la-classification-des-documents-confidentiels-et-la-protection-des-donnees-sensibles/ 

# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context

# testset = generator.generate_with_langchain_docs(documents, test_size=10)

# print(testset)

Evaluer


In [28]:
# !pip install ragas datasets

In [29]:
import pandas as pd 
from datasets import Dataset
from tqdm import tqdm
import pyarrow as py
import pyarrow as pa
import pyarrow.dataset as ds
from ragas import evaluate
from ragas.metrics import  faithfulness, answer_relevancy, context_recall, context_precision #, harmfulness
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset, Features, Sequence, Value, load_dataset


# Définir le dataset explicitement 
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Sequence(Value("string")),
    "ground_truth": Value("string")
})
dataset = Dataset.from_dict(df, features=features)

# ragas_llm = LangchainLLMWrapper(llm)

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper

# wrapper around azure_model 
ragas_azure_model = LangchainLLMWrapper(llm)
# patch the new RagasLLM instance
faithfulness.llm = ragas_azure_model
answer_relevancy.llm = ragas_azure_model
context_precision.llm = ragas_azure_model
context_recall.llm = ragas_azure_model
# harmfulness.llm = ragas_azure_model


In [31]:
# init and change the embeddings
# only for answer_relevancy
azure_embeddings = OpenAIEmbeddings(
    # deployment="text-embedding-ada-002",
    model="text-embedding-ada-002",
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_type="azure"
)
answer_relevancy.embeddings = azure_embeddings

c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\embeddings\openai.py:336: UserWarning: If you have openai>=1.0.0 installed and are using Azure, please use the `AzureOpenAIEmbeddings` class.
  warnings.warn(


In [32]:
evaluate_result = evaluate(
       dataset=dataset,
       metrics=[faithfulness, answer_relevancy,  context_recall, context_precision],
       llm=ragas_azure_model,
       raise_exceptions=False,
)

evaluate_result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 37, in sema_coro
    return await coro
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\executor.py", line 111, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\site-packages\ragas\metrics\base.py", line 125, in ascore
    raise e
  File "c:\Users\paul.peton\AppData\Local\Programs\Python\Python310\lib\s

{'faithfulness': 0.8738, 'answer_relevancy': 0.9586, 'context_recall': 0.7875, 'context_precision': 0.8687}